In [2]:
import numpy as np
import pandas as pd 

data_root = 'data/'

train_photos = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(data_root+"train_biz_HOGfeatures.csv")
test_df  = pd.read_csv(data_root+"test_biz_HOGfeatures.csv")

y_train = train_df['label'].values
X_train = train_df['feature vector'].values
X_test = test_df['feature vector'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x)>0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

y_train = np.array([convert_label_to_array(y) for y in train_df['label']])
X_train = np.array([convert_feature_to_vector(x) for x in train_df['feature vector']])
X_test = np.array([convert_feature_to_vector(x) for x in test_df['feature vector']])

In [3]:
print "X_train: ", X_train.shape
print "y_train: ", y_train.shape
print "X_test: ", X_test.shape
print "train_df:"
train_df[0:5]

X_train:  (1115, 2048)
y_train:  (1115,)
X_test:  (6196, 2048)
train_df:


,business,label,feature vector
0,1001,"(0, 1, 6, 8)","[0.26290563, 0.094085656, 0.085721985, 0.09975..."
1,100,"(1, 2, 4, 5, 6, 7)","[0.084665455, 0.042148672, 0.098702617, 0.2758..."
2,1006,"(1, 2, 4, 5, 6)","[0.066267937, 0.5848918, 0.31986001, 0.0071875..."
3,101,"(1, 2, 3, 4, 5, 6)","[0.14571482, 0.047799882, 0.06038243, 0.108079..."
4,1012,"(1, 2, 3, 5, 6)","[0.077827349, 0.038681846, 0.044559114, 0.0929..."


In [4]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

import time
t=time.time()

mlb = MultiLabelBinarizer()
y_ptrain= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_train, y_ptrain, test_size=.2,random_state=random_state)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
classifier.fit(X_ptrain, y_ptrain)

y_ppredict = classifier.predict(X_ptest)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  79.6 sec


In [5]:
print "Samples of predicted labels (in binary matrix):\n", y_ppredict[0:3]
print "\nSamples of predicted labels:\n", mlb.inverse_transform(y_ppredict[0:3])


Samples of predicted labels (in binary matrix):
[[1 1 0 1 0 1 0 0 0]
 [0 1 0 1 1 1 1 1 1]
 [0 0 0 1 1 0 0 0 1]]

Samples of predicted labels:
[(0, 1, 3, 5), (1, 3, 4, 5, 6, 7, 8), (3, 4, 8)]


In [6]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_ppredict, axis=0), len(y_ppredict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_ppredict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,49,137,144,113,56,175,191,49,125,223
biz ratio,22%,61%,65%,51%,25%,78%,86%,22%,56%,100%


In [7]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

F1 score:  0.562715199213
Individual Class F1 score:  [ 0.14678899  0.59259259  0.61710037  0.43636364  0.26771654  0.69620253
  0.79886686  0.31746032  0.53497942]
